# ESG Sentiment Analysis Demo

This notebook demonstrates the core functionality of the ESG Sentiment Scorer system.


In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Configuration and Setup

In [ ]:
# Load configuration
from config.settings import settings, ESGCategories

print("ESG Categories:")
for category, config in ESGCategories.get_all_categories().items():
    print(f"\n{category.upper()}:")
    print(f"Keywords: {config['keywords'][:5]}...")  # Show first 5 keywords

## 2. News Scraping Demo

In [ ]:
# Demo news scraping
import asyncio
from src.scraping.news_scraper import NewsScraperEngine

async def demo_news_scraping():
    async with NewsScraperEngine() as scraper:
        # Search for company news
        articles = await scraper.search_company_news("Tesla", days_back=30)
        
        print(f"Found {len(articles)} articles about Tesla:")
        for i, article in enumerate(articles, 1):
            print(f"\n{i}. {article.title}")
            print(f"   Source: {article.source}")
            print(f"   Keywords: {', '.join(article.keywords)}")
            print(f"   Date: {article.published_date}")
        
        return articles

# Run the demo
articles = await demo_news_scraping()

## 3. Sentiment Analysis Demo

In [ ]:
# Demo sentiment analysis
from src.nlp.sentiment_analyzer import ESGSentimentAnalyzer, ESGScorer

async def demo_sentiment_analysis():
    analyzer = ESGSentimentAnalyzer()
    
    # Sample texts for analysis
    sample_texts = [
        "Tesla announces major breakthrough in battery recycling technology, reducing environmental impact by 40%.",
        "Company faces criticism over workplace safety conditions and employee satisfaction scores.",
        "Board of directors approves new transparency measures and executive compensation guidelines.",
        "Investigation launched into potential environmental violations at manufacturing facility."
    ]
    
    results = []
    
    print("ESG Sentiment Analysis Results:\n")
    
    for i, text in enumerate(sample_texts, 1):
        result = await analyzer.analyze_text(text, "Tesla")
        results.append(result)
        
        print(f"Text {i}: {text[:50]}...")
        print(f"  Overall Sentiment: {result.overall_sentiment:.2f}")
        print(f"  ESG Scores: E={result.esg_scores.get('environmental', 0):.2f}, "
              f"S={result.esg_scores.get('social', 0):.2f}, "
              f"G={result.esg_scores.get('governance', 0):.2f}")
        print(f"  Key Themes: {', '.join(result.key_themes)}")
        print(f"  Risk Indicators: {', '.join(result.risk_indicators)}\n")
    
    return results

# Run sentiment analysis demo
sentiment_results = await demo_sentiment_analysis()

## 4. ESG Score Calculation

In [ ]:
# Calculate overall ESG scores
scorer = ESGScorer()
final_scores = scorer.calculate_esg_score(sentiment_results)

print("Final ESG Scores:")
for category, score in final_scores.items():
    print(f"{category.title()}: {score:.3f}")

# Create a DataFrame for visualization
scores_df = pd.DataFrame([
    {'Category': 'Environmental', 'Score': final_scores['environmental']},
    {'Category': 'Social', 'Score': final_scores['social']},
    {'Category': 'Governance', 'Score': final_scores['governance']},
])

print("\nScores DataFrame:")
print(scores_df)

## 5. Data Visualization

In [ ]:
# Create ESG score visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. ESG Scores Bar Chart
categories = ['Environmental', 'Social', 'Governance']
scores = [final_scores['environmental'], final_scores['social'], final_scores['governance']]
colors = ['#2E8B57', '#4169E1', '#DC143C']

bars = ax1.bar(categories, scores, color=colors, alpha=0.7)
ax1.set_title('ESG Scores by Category', fontsize=14, fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_ylim(0, 1)

# Add value labels on bars
for bar, score in zip(bars, scores):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Overall Score Gauge (using pie chart)
overall_score = final_scores['overall']
remaining = 1 - overall_score

wedges, texts = ax2.pie([overall_score, remaining], 
                       colors=['#2E8B57', '#E0E0E0'], 
                       startangle=90, counterclock=False)
ax2.add_patch(plt.Circle((0,0), 0.5, color='white'))
ax2.text(0, 0, f'{overall_score:.3f}\nOverall\nESG Score', 
         ha='center', va='center', fontsize=12, fontweight='bold')
ax2.set_title('Overall ESG Score', fontsize=14, fontweight='bold')

# 3. Sentiment Distribution
sentiments = [result.overall_sentiment for result in sentiment_results]
ax3.hist(sentiments, bins=5, color='skyblue', alpha=0.7, edgecolor='black')
ax3.axvline(np.mean(sentiments), color='red', linestyle='--', 
            label=f'Mean: {np.mean(sentiments):.3f}')
ax3.set_title('Sentiment Score Distribution', fontsize=14, fontweight='bold')
ax3.set_xlabel('Sentiment Score')
ax3.set_ylabel('Frequency')
ax3.legend()

# 4. Confidence Scores
confidences = [result.confidence for result in sentiment_results]
text_indices = range(1, len(confidences) + 1)

ax4.bar(text_indices, confidences, color='orange', alpha=0.7)
ax4.set_title('Analysis Confidence by Text', fontsize=14, fontweight='bold')
ax4.set_xlabel('Text Sample')
ax4.set_ylabel('Confidence Score')
ax4.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 6. Interactive Plotly Visualizations

In [ ]:
# Create interactive radar chart
categories = ['Environmental', 'Social', 'Governance', 'Transparency', 'Innovation']
values = [
    final_scores['environmental'], 
    final_scores['social'], 
    final_scores['governance'],
    final_scores['confidence'],  # Use confidence as transparency proxy
    0.75  # Mock innovation score
]

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=values,
    theta=categories,
    fill='toself',
    name='Tesla ESG Profile',
    line_color='rgba(46, 139, 87, 0.8)'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    showlegend=True,
    title="Interactive ESG Performance Radar Chart",
    title_x=0.5
)

fig.show()

## 7. Risk Assessment Summary

In [ ]:
# Aggregate risk indicators
all_risks = []
all_themes = []

for result in sentiment_results:
    all_risks.extend(result.risk_indicators)
    all_themes.extend(result.key_themes)

# Count occurrences
risk_counts = pd.Series(all_risks).value_counts()
theme_counts = pd.Series(all_themes).value_counts()

print("Risk Assessment Summary:")
print("=" * 30)
print(f"Overall ESG Score: {final_scores['overall']:.3f}")
print(f"Analysis Confidence: {final_scores['confidence']:.3f}")

print("\nTop Risk Indicators:")
if len(risk_counts) > 0:
    for risk, count in risk_counts.head().items():
        print(f"  - {risk}: {count} occurrences")
else:
    print("  - No significant risks identified")

print("\nTop ESG Themes:")
if len(theme_counts) > 0:
    for theme, count in theme_counts.head().items():
        print(f"  - {theme}: {count} occurrences")
else:
    print("  - No significant themes identified")

# Risk level assessment
overall_score = final_scores['overall']
if overall_score >= 0.7:
    risk_level = "Low Risk"
    risk_color = "🟢"
elif overall_score >= 0.5:
    risk_level = "Medium Risk"
    risk_color = "🟡"
else:
    risk_level = "High Risk"
    risk_color = "🔴"

print(f"\n{risk_color} Investment Risk Level: {risk_level}")

# Investment recommendation
if overall_score >= 0.7:
    recommendation = "STRONG BUY - Excellent ESG profile"
elif overall_score >= 0.6:
    recommendation = "BUY - Good ESG fundamentals"
elif overall_score >= 0.4:
    recommendation = "HOLD - Monitor ESG developments"
else:
    recommendation = "AVOID - Significant ESG concerns"

print(f"📈 Investment Recommendation: {recommendation}")

## 8. Export Results

In [ ]:
# Create comprehensive results DataFrame
results_data = []

for i, result in enumerate(sentiment_results):
    results_data.append({
        'text_id': i + 1,
        'overall_sentiment': result.overall_sentiment,
        'confidence': result.confidence,
        'environmental_score': result.esg_scores.get('environmental', 0),
        'social_score': result.esg_scores.get('social', 0),
        'governance_score': result.esg_scores.get('governance', 0),
        'key_themes': ', '.join(result.key_themes),
        'risk_indicators': ', '.join(result.risk_indicators),
        'language': result.language
    })

results_df = pd.DataFrame(results_data)
print("Detailed Results:")
print(results_df)

# Save results
results_df.to_csv('../data/processed/esg_analysis_results.csv', index=False)
print("\nResults saved to ../data/processed/esg_analysis_results.csv")

# Save final scores
final_scores_df = pd.DataFrame([final_scores])
final_scores_df.to_csv('../data/processed/final_esg_scores.csv', index=False)
print("Final scores saved to ../data/processed/final_esg_scores.csv")

## Conclusion

This notebook demonstrated the core functionality of the ESG Sentiment Scorer:

1. **News Scraping**: Retrieved relevant articles about companies
2. **Sentiment Analysis**: Analyzed text for ESG-related sentiment
3. **ESG Classification**: Categorized content into Environmental, Social, and Governance factors
4. **Risk Assessment**: Identified potential risk indicators
5. **Scoring**: Calculated comprehensive ESG scores
6. **Visualization**: Created interactive dashboards and charts
7. **Investment Recommendations**: Generated data-driven investment advice

The system provides a solid foundation for ESG-based investment analysis and can be extended with additional features like real-time monitoring, multi-language support, and advanced ML models.